In [1]:
import networkx as nx
import numpy as np
from src.diffusion_models import independent_cascade_model, linear_threshold_model
from src.polarization import create_polarized_graph, optimize_tau, random_color_graph
from src.seed import seed_random, seed_degree

In [2]:

def add_edges_using_optimize_tau(G, C, k):
    """Add edges to the graph G using the optimize_tau function."""
    F_set, _ = optimize_tau(C, G, k)
    G.add_edges_from(F_set)

def run_diffusion_with_same_seed(create_graph_func, num_nodes, intra_group_connectness, inter_group_connectness, add_edges_func, diffusion_model_func, seed_func, seed_count, threshold=None, k=5):
    # Create original graph
    original_graph = create_graph_func(num_nodes, intra_group_connectness, inter_group_connectness)
    # color the graph
    random_color_graph(original_graph)

    # Select seeds
    seeds = seed_func(original_graph, seed_count)

    # Generate thresholds for each node if using linear threshold model
    if diffusion_model_func == linear_threshold_model:
        thresholds = {node: np.random.uniform(0, 1) for node in original_graph.nodes}
    else:
        thresholds = threshold

    # Add new edges to the graph
    modified_graph = original_graph.copy()
    add_edges_func(modified_graph, C=seeds, k=k)  # Assuming k=5 for adding 5 edges

    print("Original graph")
    print("Number of nodes: ", original_graph.number_of_nodes())
    print("Number of edges: ", original_graph.number_of_edges())

    print("Modified graph")
    print("Number of nodes: ", modified_graph.number_of_nodes())
    print("Number of edges: ", modified_graph.number_of_edges())
    

    # Run diffusion on original graph
    result_original = diffusion_model_func(original_graph, seeds, thresholds)

    # Run diffusion on modified graph
    result_modified = diffusion_model_func(modified_graph, seeds, thresholds)

    return result_original, result_modified

In [15]:
result_original, result_modified = run_diffusion_with_same_seed(
    create_polarized_graph, num_nodes=50, intra_group_connectness=0.2, inter_group_connectness=0.01,
    add_edges_func=add_edges_using_optimize_tau, diffusion_model_func=linear_threshold_model, 
    seed_func=seed_random, seed_count=10, threshold=0.5, k = 27
)
result_original[0], result_modified[0] 

100%|██████████| 27/27 [00:00<00:00, 168.39it/s]

Original graph
Number of nodes:  50
Number of edges:  235
Modified graph
Number of nodes:  50
Number of edges:  262


(12, 15)